In [1]:
import sys, os
import requests
import pandas as pd

In [2]:
import numpy as np
from dotenv import load_dotenv

In [3]:
from pathlib import Path
load_dotenv()
print(".env loaded (if present)")

.env loaded (if present)


In [4]:
import os
api_key = os.getenv("OPENAI_API_KEY")
print(api_key)  # 验证是否成功加载

J1O8X0E685L3S8PD


In [5]:
def get_alpha_vantage_data(symbol, function='TIME_SERIES_DAILY', output_size='compact'):

    base_url = "https://www.alphavantage.co/query"
    
    params = {
        "function": function,
        "symbol": symbol,
        "outputsize": output_size,
        "apikey": api_key,
        "datatype": "json" 
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  
        data = response.json()
        
        if 'Time Series (Daily)' in data:
            time_series = data['Time Series (Daily)']
        elif 'Weekly Time Series' in data:
            time_series = data['Weekly Time Series']
        elif 'Monthly Time Series' in data:
            time_series = data['Monthly Time Series']
        else:
            raise ValueError("无法解析返回的数据格式")
        
        df = pd.DataFrame.from_dict(time_series, orient='index')
        df.index = pd.to_datetime(df.index)  
        df = df.astype(float)  
        
        df.columns = [col.split('. ')[1] if '. ' in col else col for col in df.columns]
        
        df = df.sort_index(ascending=True)
        
        return df
    
    except Exception as e:
        print(f"获取数据时出错: {e}")
        return None


In [6]:
aapl_data = get_alpha_vantage_data('AAPL', function='TIME_SERIES_DAILY', output_size='compact')

In [7]:
aapl_data = aapl_data.dropna(axis=0, how='any')

In [8]:
aapl_data

,open,high,low,close,volume
2025-03-25,220.770,224.10,220.0800,223.75,34493583.0
2025-03-26,223.510,225.02,220.4700,221.53,34532656.0
2025-03-27,221.390,224.99,220.5601,223.85,37094774.0
2025-03-28,221.670,223.81,217.6800,217.90,39818617.0
2025-03-31,217.005,225.62,216.2300,222.13,65299321.0
...,...,...,...,...,...
2025-08-11,227.920,229.56,224.7600,227.18,61806132.0
2025-08-12,228.005,230.80,227.0700,229.65,55672301.0
2025-08-13,231.070,235.00,230.4300,233.33,69878546.0
2025-08-14,234.055,235.12,230.8500,232.78,51916275.0


In [9]:
aapl_data.to_csv("C://Users//Zhangchensi//bootcamp_Chensi_Zhang//data//raw//api_pull.csv")

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# 示例：爬取维基百科的简单表格
url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# 定位表格（通常用find('table')或特定class）
table = soup.find('table', {'class': 'wikitable'})

# 解析为DataFrame
df = pd.read_html(str(table))[0] 
df

C:\Users\Zhangchensi\AppData\Local\Temp\ipykernel_27688\1848354138.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


Country/Territory IMF[1][12]            World Bank[13]             \
    Country/Territory   Forecast       Year       Estimate       Year   
0               World  113795678       2025      111326370       2024   
1       United States   30507217       2025       29184890       2024   
2               China   19231705  [n 1]2025       18743803  [n 3]2024   
3             Germany    4744804       2025        4659929       2024   
4               India    4187017       2025        3912686       2024   
..                ...        ...        ...            ...        ...   
217          Kiribati        312       2025            308       2024   
218  Marshall Islands        297       2025            280       2024   
219             Nauru        169       2025            160       2024   
220        Montserrat          —          —              —          —   
221            Tuvalu         65       2025             62       2023   

    United Nations[14]             
              Estimate       Year  
0            100834796       2022  
1             27720700       2023  
2             17794782  [n 1]2023  
3              4525704       2023  
4              3575778       2023  
..                 ...        ...  
217                289       2023  
218                270       2023  
219                176       2023  
220                 80       2023  
221                 68       2023  

[222 rows x 7 columns]

In [11]:
df.columns = df.columns.droplevel(0)
cols = pd.Series(df.columns)
for dup in cols[cols.duplicated()].unique():
    dup_mask = cols == dup
    dup_idx = dup_mask.cumsum() 
    cols.loc[dup_mask] = cols.loc[dup_mask] + '_' + dup_idx.loc[dup_mask].astype(str)
df.columns = cols
df

,Country/Territory,Forecast,Year_1,Estimate_1,Year_2,Estimate_2,Year_3
0,World,113795678,2025,111326370,2024,100834796,2022
1,United States,30507217,2025,29184890,2024,27720700,2023
2,China,19231705,[n 1]2025,18743803,[n 3]2024,17794782,[n 1]2023
3,Germany,4744804,2025,4659929,2024,4525704,2023
4,India,4187017,2025,3912686,2024,3575778,2023
...,...,...,...,...,...,...,...
217,Kiribati,312,2025,308,2024,289,2023
218,Marshall Islands,297,2025,280,2024,270,2023
219,Nauru,169,2025,160,2024,176,2023
220,Montserrat,—,—,—,—,80,2023


In [12]:
df = df.rename(columns={'Forecast': 'IMF_Forecast', 'Year_1': 'IMF_Year','Estimate_1': 'WB_Estimate','Year_2': 'WB_Year','Estimate_2': 'UN_Estimate','Year_3': 'UN_Year'})
df = df.dropna(axis=0, how='any')

In [13]:
df

,Country/Territory,IMF_Forecast,IMF_Year,WB_Estimate,WB_Year,UN_Estimate,UN_Year
0,World,113795678,2025,111326370,2024,100834796,2022
1,United States,30507217,2025,29184890,2024,27720700,2023
2,China,19231705,[n 1]2025,18743803,[n 3]2024,17794782,[n 1]2023
3,Germany,4744804,2025,4659929,2024,4525704,2023
4,India,4187017,2025,3912686,2024,3575778,2023
...,...,...,...,...,...,...,...
217,Kiribati,312,2025,308,2024,289,2023
218,Marshall Islands,297,2025,280,2024,270,2023
219,Nauru,169,2025,160,2024,176,2023
220,Montserrat,—,—,—,—,80,2023


In [14]:
df.to_csv("C://Users//Zhangchensi//bootcamp_Chensi_Zhang//data//raw//scrape.csv")